# Reducing Risk: A Data-Driven Approach to Traffic Accident Mitigation in the U.S.



## 1. Overview

**1.1 Bottom Line Up Front:**
This project analyzes X.

**1.2 Key Early Insights:**
TBD

**1.3 Dataset Overview:**

TBD

Citations TBD

## 2. Business Understanding

**2.1 Relevance of the Data**

X

**2.2 Analytical Questions**

X

**2.3 Goals & Benefits**

X

**2.4 Stakeholders**

X



## 3. Data Understanding

###3.1 Data and Library Loading:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load relevant imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as stats
from scipy.stats import ttest_ind
import statsmodels

###3.2 Identify Data Quality Issues

## 4. Data Preparation

###4.1 Define functions 1-5 to:

1. Drop unecessary columns
2. Parse datetimes, drop invalids, create duration
3. Remove outliers
4. Handle missing values
5. Feature engineering

###4.2 Call functions and summarize data:

## 5. EDA

### 0. Basic Exploration:

### Question 1: 


### Question 2:


## 6. Model Development

## 7. Evaluation: Business Insights and Recommendations


## 8. Conclusion and Next Steps